# Visualizar mapas de Chile

In [5]:
import pandas as pd
from pandasql import sqldf
import geopandas as gpd

In [6]:
FILE = "./noticias_losrios_2022_porcomuna_sentimientos.csv"
DATASET = pd.read_csv(FILE)
DATASET.drop('Unnamed: 0', axis=1, inplace=True)
DATASET = DATASET.dropna(subset=['text'])

DATASET

,id_news,country,media_outlet,url,title,text,date,search,Lago_Ranco,Los_Lagos,...,La_Unión,Panguipulli,Paillaco,Corral,Máfil,Lanco,Mariquina,Valdivia,prediction,score
0,47314428.0,chile,suractual,https://www.suractual.cl/2022/01/24/ante-proye...,Ante proyecto Tierras Raras municipio y Unión ...,“Considerando la evolución que ha tenido el Pr...,2022-01-24,NaN,0,0,...,0,0,0,0,0,0,0,0,NEU,0.988633
1,47719086.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/25/trac...,Tractocamión volcó esta mañana en puente Lloll...,El accidente se produjo pasadas las 06:00 AM y...,2022-01-25,NaN,0,0,...,1,0,0,0,0,0,0,0,NEU,0.998064
2,47719110.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/27/gobe...,Gobernadores Regionales de Los Ríos y Los Lago...,La instancia buscó generar un trabajo colabora...,2022-01-27,NaN,0,1,...,0,0,0,0,0,0,0,0,NEU,0.931461
3,47700339.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Alegan inocencia de acusadas por cruel asesina...,La defensa de las dos mujeres acusadas de enca...,2022-01-05,NaN,0,0,...,0,0,0,0,0,0,0,1,NEU,0.973683
4,47700425.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Destacan ejecución del Programa Vínculos en Lo...,Para dar protección y acompañamiento a adultos...,2022-01-03,NaN,0,0,...,0,0,0,0,0,0,0,1,POS,0.536478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15142,50428917.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/educacion...,Admisión 2023: Caravana Técnico Profesional de...,Desde este lunes 12 y hasta el miércoles 14 de...,2022-12-13,NaN,0,1,...,0,0,0,0,0,0,0,0,NEU,0.979722
15143,50428943.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/emergenci...,Triple colisión en Los Lagos deja al menos un ...,Tres vehículos se vieron involucrados en un ac...,2022-12-07,NaN,0,1,...,0,0,0,0,0,0,0,0,NEG,0.716544
15144,50428975.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/opinion/2...,Columna de opinión: No quedó ni el sombrero - ...,“Somos en realidad sacos llenos de cosas inédi...,2022-12-09,NaN,0,0,...,0,0,0,0,0,0,0,0,NEG,0.977032
15145,50429002.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/emergenci...,Triste despertar en Los Lagos: comunidad lamen...,Un triste despertar ha tenido este domingo la ...,2022-12-04,NaN,0,1,...,0,0,0,0,0,0,0,0,NEG,0.997505


In [7]:
SHAPEFILE=r'./shape/comunas.shp' #shapefile de las regiones y comunas de Chile
REGION = 14 #Los Rios

# fuente: https://www.bcn.cl/siit/mapas_vectoriales

In [9]:
territory = gpd.read_file(SHAPEFILE, encoding='utf-8')
territory = territory[territory["codregion"] == REGION]
# Obtenemos los datos de las Columnas "Comuna", "geometry"
territory = territory[["Comuna", "geometry"]]

In [10]:
def save_map(df, territory, image_name: str = "Mapa_Calor"):
    
    #Junta las tablas df y Pais
    territory = territory.merge(df, on = "Comuna")
    
    #Inicializa matplotlib para la creacion del mapa
    fig, ax = plt.subplots(figsize=(10, 10))
    max_presente = df.max()[1]#valore maximo para la barra de calor
    territory.plot(column = 'Valor',ax=ax,edgecolor = "gray", cmap = 'OrRd', legend = False).set_axis_off()
    bar_info = plt.cm.ScalarMappable(cmap="Reds", norm=plt.Normalize(vmin=0,vmax=max_presente))
    bar_info._A = []
    cbar = fig.colorbar(bar_info)
    
    #Guardar imagen en nombre_imagen
    plt.savefig('.//{}.svg'.format(image_name), bbox_inches = "tight",transparent = True)

In [11]:
pysqldf = lambda q: sqldf(q, globals())

query = """
SELECT sum("La_Unión") as "La Unión",
sum("Los_Lagos")as "Los Lagos",
sum(Máfil)as Máfil,
sum(Paillaco) as Paillaco,
sum(Lanco) as Lanco,
sum(Panguipulli) as Panguipulli,
sum("Río_Bueno") as "Río Bueno",
sum(Mariquina) as Mariquina,
sum(Corral) as Corral,
sum(Valdivia) as Valdivia,
sum("Lago_Ranco") as "Lago Ranco",
sum(Futrono) as Futrono 
FROM DATASET
"""

print("Cantidad de noticias total en 2022 por comuna")
query1=pysqldf(query)
query1

Cantidad de noticias total en 2022 por comuna


,La Unión,Los Lagos,Máfil,Paillaco,Lanco,Panguipulli,Río Bueno,Mariquina,Corral,Valdivia,Lago Ranco,Futrono
0,1967,1729,498,2026,778,1271,1352,701,732,8415,838,1605


In [12]:
df=query1

lista =[]
cont = 0
for i in territory["Comuna"]:
    print(i)
    lista.append({"Comuna": i, "Valor": df.iloc[0][cont]})
    cont +=1
df = pd.DataFrame(lista)

save_map(df, territory, "mapa")
plt.savefig("mapa.png", dpi=500)

La Unión
Los Lagos
Máfil
Paillaco
Lanco
Panguipulli
Río Bueno
Mariquina
Corral
Valdivia
Lago Ranco
Futrono


NameError: name 'pandas' is not defined